# Banchmark fully off-policy DQN on Cartpole

Is stable-baselines **Behavior Cloning** the right choice to make an off-policy dataset?
* The dokumentation says: _"for a given observation, the action taken by the policy must be the one taken by the expert"_
* That´s in my opption not the same as filling a replay buffer.

In [2]:
from stable_baselines import DQN

### Collecting off-policy data to tranin DQN

The dataset is saved as expert_cartpole.npz in this folder.

In [ ]:
from stable_baselines.gail import generate_expert_traj

model = DQN('MlpPolicy', 'CartPole-v1', verbose=0)
      # Train a DQN agent for 1e5 timesteps and generate 10 trajectories
      # data will be saved in a numpy archive named `expert_cartpole.npz`
generate_expert_traj(model, 'expert_cartpole', n_timesteps=int(1e6), n_episodes=int(1e4))

### Traning DQN with fully off-policy data

* I´m note sure if `model.pretrain()` is the same like a `reaply_buffer`

In [3]:
from stable_baselines.gail import ExpertDataset
# Using only one expert trajectory
# you can specify `traj_limitation=-1` for using the whole dataset
dataset = ExpertDataset(expert_path='expert_cartpole.npz',
                        traj_limitation=-1, batch_size=128)

model = DQN('MlpPolicy', 'CartPole-v1', verbose=1)
# Pretrain the DQN
model.pretrain(dataset, n_epochs=100)

# Test the pre-trained model
env = model.get_env()
obs = env.reset()

reward_sum = 0.0
for _ in range(1000):
        action, _ = model.predict(obs)
        obs, reward, done, _ = env.step(action)
        reward_sum += reward
#         env.render()
        if done:
                print(reward_sum)
                reward_sum = 0.0
                obs = env.reset()

env.close()

actions (1054, 1)
obs (1054, 4)
rewards (1054,)
episode_returns (100,)
episode_starts (1054,)
Total trajectories: -1
Total transitions: 1054
Average returns: 10.54
Std for returns: 2.5472337937456784
Creating environment from the given name, wrapped in a DummyVecEnv.
Pretraining with Behavior Cloning...
==== Training progress 10.00% ====
Epoch 10
Training loss: 0.562942, Validation loss: 0.561478

==== Training progress 20.00% ====
Epoch 20
Training loss: 0.400825, Validation loss: 0.408492

==== Training progress 30.00% ====
Epoch 30
Training loss: 0.279523, Validation loss: 0.286960

==== Training progress 40.00% ====
Epoch 40
Training loss: 0.215821, Validation loss: 0.228228

==== Training progress 50.00% ====
Epoch 50
Training loss: 0.180851, Validation loss: 0.179090

==== Training progress 60.00% ====
Epoch 60
Training loss: 0.157219, Validation loss: 0.157848

==== Training progress 70.00% ====
Epoch 70
Training loss: 0.144404, Validation loss: 0.145440

==== Training progress 